## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2118. Build the Equation (Hard)**

**Table: Terms**

| Column Name | Type |
|-------------|------|
| power       | int  |
| factor      | int  |

power is the column with unique values for this table.
Each row of this table contains information about one term of the equation.
power is an integer in the range [0, 100].
factor is an integer in the range [-100, 100] and cannot be zero.
 
You have a very powerful program that can solve any equation of one variable in the world. The equation passed to the program must be formatted as follows:
- The left-hand side (LHS) should contain all the terms.
- The right-hand side (RHS) should be zero.
- Each term of the LHS should follow the format "<sign><fact>X^<pow>" where:
  - <sign> is either "+" or "-".
  - <fact> is the absolute value of the factor.
  - <pow> is the value of the power.
- If the power is 1, do not add "^<pow>".
  - For example, if power = 1 and factor = 3, the term will be "+3X".
- If the power is 0, add neither "X" nor "^<pow>".
  - For example, if power = 0 and factor = -3, the term will be "-3".
- The powers in the LHS should be sorted in descending order.

**Write a solution to build the equation.**

The result format is in the following example.

**Example 1:**

**Input:** 

**Terms table:**

| power | factor |
|-------|--------|
| 2     | 1      |
| 1     | -4     |
| 0     | 2      |

**Output:** 
| equation     |
|--------------|
| +1X^2-4X+2=0 |

**Example 2:**

**Input:** 

**Terms table:**

| power | factor |
|-------|--------|
| 4     | -4     |
| 2     | 1      |
| 1     | -1     |

**Output:** 
| equation        |
|-----------------|
| -4X^4+1X^2-1X=0 |
 
**Follow up:** What will be changed in your solution if the power is not a primary key but each power should be unique in the answer?

In [0]:
terms_data_2118 = [
    (2, 1),
    (1, -4),
    (0, 2)
]

terms_columns_2118 = ["power", "factor"]

terms_df_2118 = spark.createDataFrame(terms_data_2118, terms_columns_2118)
terms_df_2118.show()

+-----+------+
|power|factor|
+-----+------+
|    2|     1|
|    1|    -4|
|    0|     2|
+-----+------+



In [0]:
df_terms_2118 = terms_df_2118\
                    .withColumn( "term",
                                concat(
                                    when(col("factor") > 0, lit("+")).otherwise(lit("")),col("factor"),
                                    when(col("power") == 0, lit(""))
                                    .when(col("power") == 1, lit("X"))
                                    .otherwise(concat(lit("X^"), col("power")))
                                    )
                                )

In [0]:
df_terms_2118 = df_terms_2118.orderBy(col("power").desc())

In [0]:
df_terms_2118\
    .select(
        concat_ws("", collect_list("term")).alias("lhs"))\
            .withColumn("equation", concat(col("lhs"), lit("=0"))) \
                .select("equation").show()

+------------+
|    equation|
+------------+
|+1X^2-4X+2=0|
+------------+

